In [ ]:
import boto3
import json
import uuid

# Initialize the Bedrock Agent client (replace region as appropriate)
client = boto3.client('bedrock-agent-runtime', region_name='eu-west-2')

# Invoke the agent (replace agent IDs and input text as needed)
response = client.invoke_agent(
    agentId='5NSQXINYPO',
    agentAliasId='CNEFXKBDCN',
    sessionId=str(uuid.uuid4()),  # Now uuid is imported 
    inputText="Is Omeprazole likely to be carcinogenic in rodent species??",
    sessionState={
        "sessionAttributes": {
            "system": """
Show the generated SQL query .           
Use the column 'experimenttime_in_years' when using "WHERE" SQL CLAUSE when reffering to 'experiment time' or to any duration reffrence: months, days, years, month, day, year.
When using 'experimenttime_in_years' in comparation queris use it like this: 'experimenttime_in_years::NUMERIC'.
When using 'experimenttime_in_years' use NESTED SELECT FROM the table vitic_lhasa.carcinogenicity like 'SELECT * FROM (SELECT * FROM vitic_lhasa.carcinogenicity WHERE experimenttime_in_years is NOT NULL) WHERE experimenttime_in_years::NUMERIC LIMIT 100' 
When the question reffers to 'rodent' lookup the 'species' column from the vitic_lhasa.carcinogenicity table for the values: 'Mouse', 'Rat', 'Hamster'.
When the Question reffers to 'similar compounds' please use the following 'tanimoto_sml(rdkit_fp(mol_from_smiles('smiles_convert from the vitic_lhasa.structure table column'::cstring)), fingerprint) as similarity_index from VITIC_LHASA.structureS'
The column testtype of the vitic_lhasa.carcinogenicity table is a human-readable description of the test.
When the Question reffers to 'test type' use vector similarity search against the column 'embedding' by using a WHERE clause like WHERE to_tsvector('english', testtype) @@ plainto_tsquery('english', 'testtype').
When the Question reffers to 'frequency' use vector similarity search against the column 'embedding_frequency' by using a WHERE clause like WHERE to_tsvector('english', frequency) @@ plainto_tsquery('english', 'frequency').
For 'frequency' the '1/n periods' value means once every 'n' periods. Example: '1/3 days', '1/3 weeks', '1/3 months', '1/3 years'
For 'frequency' the '1 per period'we have examples like: '1/day', '1/week', '1/month','1/year'. 
When the Question reffers to carcinogenic' it means that the column result from the table vitic_lhasa.carcinogenicity is 'Positive'."""
        }
    },
    enableTrace=False,
    endSession=False
)

# The 'completion' field is a streaming EventStream
event_stream = response.get('completion')
full_response = ""  # to accumulate the output text

# Iterate over the streamed events
for event in event_stream:
    # Each event with a 'chunk' contains part of the answer
    if 'chunk' in event:
        bytes_chunk = event['chunk'].get('bytes')
        if bytes_chunk:
            # Decode bytes to string (UTF-8)
            chunk_str = bytes_chunk.decode('utf-8')
            full_response += chunk_str
    elif 'exception' in event:
        # Handle error events if any (optional)
        error_info = event['exception']
        raise RuntimeError(f"Agent returned an error: {error_info}")

# At this point, full_response contains the complete response (as a JSON string)
print("Raw agent output:", full_response)

# If the output is JSON, parse it to extract the answer text (optional)
try:
    output_json = json.loads(full_response)
    # Bedrock streaming responses use a JSON structure with a 'type'
    if output_json.get("type") == "content_block_delta":
        answer_text = output_json.get("delta", {}).get("text", "")
    else:
        answer_text = full_response  # not the expected structured chunk
except json.JSONDecodeError:
    answer_text = full_response  # already plain text

print("Final answer text:", answer_text)